# Replicating medical cannabis laws and opioid overdose mortality study

[Bachhuber et al. (2014)](https://jamanetwork.com/journals/jamainternalmedicine/fullarticle/1898878) found that from 1999 to 2010 states with medical cannabis laws experienced slower increases in opioid analgesic overdose mortality.

[Shover et al. (2019)](https://www.pnas.org/doi/suppl/10.1073/pnas.1903434116#executive-summary-abstract) extend Bachhuber et al.’s analysis through 2017. They find that results from the original analysis do not hold over the longer period, and the association between state medical cannabis laws and opioid overdose mortality reversed direction from −21% to +23% and remained positive after accounting for recreational cannabis laws.

## Empirical Exercise in Stata

Note: The data used in the original article is public.

### Set Stata Magic in Python

In [10]:
%%capture
import os
os.chdir('/Program Files/Stata17/utilities')
from pystata import config
config.init('se')

### Import Data from Web

In [15]:
%%stata
* Import data
 import delimited "C:\Users\edloaeza\Documents\GitHub\Marijuana\pnas.1903434116.sd01.csv", clear


. * Import data
.  import delimited "C:\Users\edloaeza\Documents\GitHub\Marijuana\pnas.19034341
> 16.sd01.csv", clear
(encoding automatically selected: ISO-8859-1)
(25 vars, 947 obs)

. 


In [16]:
%%stata
des


Contains data
 Observations:           947                  
    Variables:            25                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
state           str14   %14s                  
year            int     %8.0g                 Year
deaths          int     %8.0g                 Deaths
population      long    %12.0g                Population
crude_rate      float   %9.0g                 Crude_Rate
age_adjusted_~e float   %9.0g                 Age_Adjusted_Rate
ag~wer_95__conf float   %9.0g                 Age_Adjusted_Rate_Lower_95__Conf
ag~per_95__conf float   %9.0g                 Age_Adjusted_Rate_Upper_95__Conf
ln_crude_mort~e float   %9.0g                 
crude_rate_lo~e float   %9.0g                 Crude_Rate_Lower_95__Confidence
crude_rate_up~e f

### Create initial variables

In [17]:
%%stata
sort state year
egen state_id=group( state)
xtset state_id year



. sort state year

. egen state_id=group( state)

. xtset state_id year

Panel variable: state_id (unbalanced)
 Time variable: year, 1999 to 2017, but with a gap
         Delta: 1 unit

. 


### Estimates for the original 1999–2010 time period

In [25]:
%%stata
gl origcontrols unemployment rxdmp_original rxid_original pmlaw_original
eststo clear
reghdfe ln_age_mort_rate medical_cannabis_law $origcontrols if year<=2010, a(state_id year) vce(r)
est store reg1
esttab reg1


. gl origcontrols unemployment rxdmp_original rxid_original pmlaw_original

. eststo clear

. reghdfe ln_age_mort_rate medical_cannabis_law $origcontrols if year<=2010, a(
> state_id year) vce(r)
(MWFE estimator converged in 5 iterations)

HDFE Linear regression                            Number of obs   =        575
Absorbing 2 HDFE groups                           F(   5,    509) =       3.77
                                                  Prob > F        =     0.0023
                                                  R-squared       =     0.8665
                                                  Adj R-squared   =     0.8494
                                                  Within R-sq.    =     0.0342
                                                  Root MSE        =     0.2914

------------------------------------------------------------------------------
             |               Robust
ln_age_mor~e | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-----------

### Using the full 1999–2017 dataset

In [26]:
%%stata
gl controls unemployment rxdmp_update pmlaw_update rxid_update
eststo clear
reghdfe ln_age_mort_rate medical_cannabis_law $controls, a(state_id year) vce(r)
est store reg2
esttab reg2


. gl controls unemployment rxdmp_update pmlaw_update rxid_update

. eststo clear

. reghdfe ln_age_mort_rate medical_cannabis_law $controls, a(state_id year) vce
> (r)
(MWFE estimator converged in 5 iterations)

HDFE Linear regression                            Number of obs   =        908
Absorbing 2 HDFE groups                           F(   5,    835) =       6.94
                                                  Prob > F        =     0.0000
                                                  R-squared       =     0.8169
                                                  Adj R-squared   =     0.8012
                                                  Within R-sq.    =     0.0422
                                                  Root MSE        =     0.3516

------------------------------------------------------------------------------
             |               Robust
ln_age_mor~e | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+-------------------------

In [12]:
%%stata
esttab IV1 IV2 IV3, label se star(* 0.10 ** 0.05 *** 0.01) ///
title(2SLS Estimates of the Demand of Cigarrets Using Panel Data for States in USA) ///
stats(IV fs Jtest pvalue N, ///
label("Instr. Variables" "F-statistic" "J-test" "P-value" "Observations"))


. esttab IV1 IV2 IV3, label se star(* 0.10 ** 0.05 *** 0.01) ///
> title(2SLS Estimates of the Demand of Cigarrets Using Panel Data for States i
> n USA) ///
> stats(IV fs Jtest pvalue N, ///
> label("Instr. Variables" "F-statistic" "J-test" "P-value" "Observations"))

2SLS Estimates of the Demand of Cigarrets Using Panel Data for States in USA
--------------------------------------------------------------------
                              (1)             (2)             (3)   
                         dlpackpc        dlpackpc        dlpackpc   
--------------------------------------------------------------------
dlavgprs                   -0.938***       -1.343***       -1.202***
                          (0.201)         (0.221)         (0.191)   

dlperinc                    0.526           0.428           0.462   
                          (0.329)         (0.289)         (0.300)   

Constant                    0.209*          0.450***        0.367***
                          (0.